<a href="https://colab.research.google.com/github/Aditya-21052131/Automating-Sales-Reports-Using-Advanced-Excel/blob/main/Automating_Sales_Reports_Using_Advanced_Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas openpyxl xlsxwriter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 1.7 MB/s eta 0:00:00


In [2]:
import sqlite3
import pandas as pd

# Create a SQLite database connection
conn = sqlite3.connect('sales_data.db')
cursor = conn.cursor()

# Create a table with sales data
cursor.execute('''
CREATE TABLE IF NOT EXISTS daily_sales (
    sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_id INTEGER,
    product_name TEXT,
    quantity_sold INTEGER,
    sale_amount REAL,
    sale_date DATE
)
''')

# Insert sample data into the sales table
data = [
    (101, 'Product A', 10, 1500.50, '2024-09-10'),
    (102, 'Product B', 5, 750.00, '2024-09-10'),
    (101, 'Product A', 20, 3000.00, '2024-09-11'),
    (103, 'Product C', 15, 2250.00, '2024-09-11'),
    (102, 'Product B', 10, 1500.00, '2024-09-12'),
    (104, 'Product D', 8, 1200.00, '2024-09-12')
]

cursor.executemany('INSERT INTO daily_sales (product_id, product_name, quantity_sold, sale_amount, sale_date) VALUES (?, ?, ?, ?, ?)', data)
conn.commit()

# Extract sales data from the database using SQL query
query = "SELECT * FROM daily_sales WHERE sale_date = '2024-09-10'"
df_sales = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the sales data
df_sales.head()


,sale_id,product_id,product_name,quantity_sold,sale_amount,sale_date
0,1,101,Product A,10,1500.5,2024-09-10
1,2,102,Product B,5,750.0,2024-09-10


In [3]:
# Check for missing values
print("Missing Values:\n", df_sales.isnull().sum())

# Summary of sales data
print(df_sales.describe())

# Add a calculated field for Total Revenue
df_sales['total_revenue'] = df_sales['quantity_sold'] * df_sales['sale_amount']

# Display the updated DataFrame
df_sales.head()


Missing Values:
 sale_id          0
product_id       0
product_name     0
quantity_sold    0
sale_amount      0
sale_date        0
dtype: int64
        sale_id  product_id  quantity_sold  sale_amount
count  2.000000    2.000000       2.000000     2.000000
mean   1.500000  101.500000       7.500000  1125.250000
std    0.707107    0.707107       3.535534   530.683639
min    1.000000  101.000000       5.000000   750.000000
25%    1.250000  101.250000       6.250000   937.625000
50%    1.500000  101.500000       7.500000  1125.250000
75%    1.750000  101.750000       8.750000  1312.875000
max    2.000000  102.000000      10.000000  1500.500000


,sale_id,product_id,product_name,quantity_sold,sale_amount,sale_date,total_revenue
0,1,101,Product A,10,1500.5,2024-09-10,15005.0
1,2,102,Product B,5,750.0,2024-09-10,3750.0


In [4]:
import openpyxl
from openpyxl.styles import Font

# Export DataFrame to Excel using XlsxWriter
output_path = '/content/sales_report_2024_09_10.xlsx'
df_sales.to_excel(output_path, index=False, engine='openpyxl')

# Open the workbook and sheet for formatting
wb = openpyxl.load_workbook(output_path)
ws = wb.active

# Apply formatting (bold headers)
for col in ws.iter_cols(min_row=1, max_row=1, min_col=1, max_col=ws.max_column):
    for cell in col:
        cell.font = Font(bold=True)

# Adjust column width
for column in ws.columns:
    max_length = 0
    column = list(column)
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2)
    ws.column_dimensions[column[0].column_letter].width = adjusted_width

# Save the formatted Excel file
wb.save(output_path)

print(f'Sales report has been saved to {output_path}')


Sales report has been saved to /content/sales_report_2024_09_10.xlsx


In [5]:
def generate_sales_report(sale_date):
    # Connect to the database
    conn = sqlite3.connect('sales_data.db')

    # SQL query to fetch data for a specific date
    query = f"SELECT * FROM daily_sales WHERE sale_date = '{sale_date}'"
    df_sales = pd.read_sql_query(query, conn)

    # Clean the data
    df_sales['total_revenue'] = df_sales['quantity_sold'] * df_sales['sale_amount']

    # Export to Excel
    output_path = f'/content/sales_report_{sale_date}.xlsx'
    df_sales.to_excel(output_path, index=False, engine='openpyxl')

    # Open workbook for formatting
    wb = openpyxl.load_workbook(output_path)
    ws = wb.active

    # Apply bold formatting to headers
    for col in ws.iter_cols(min_row=1, max_row=1, min_col=1, max_col=ws.max_column):
        for cell in col:
            cell.font = Font(bold=True)

    # Adjust column widths
    for column in ws.columns:
        max_length = 0
        column = list(column)
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column[0].column_letter].width = adjusted_width

    # Save the file
    wb.save(output_path)

    # Close the database connection
    conn.close()

    print(f'Sales report for {sale_date} has been saved to {output_path}')

# Generate report for 2024-09-10
generate_sales_report('2024-09-10')


Sales report for 2024-09-10 has been saved to /content/sales_report_2024-09-10.xlsx
